# api 크롤링

기존의 크롤링 방식은 

1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium)

혹은 

2. 파이썬 자체적으로 호출을 요청해서(requests)

데이터를 가지고 오는 방식이었습니다.

이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.

따라서 서비스 제공자 측에서는 서버 부하를 줄이고, 사용자에게 맞춤형의 데이터를 제공하기 위해 api서버를 운영합니다.

api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.)

필요없는 동영상 자료나 그림 자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.


api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.

In [9]:
# 사이트에 자료 요청
import urllib.request
#json 데이터 핸들링
import json
# DataFrame 자료형 활용
import pandas as pd 
# json 데이터를 pandas DataFrame으로 변환
from pandas.io.json import json_normalize

# 영진위 api 신청

https://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

1. 접속 후 가입 및 로그인 

2. 키발급 탭에서 키 발급하기 

3. 요청 api종류 보고 요청양식 및 데이터 확인하기 

In [10]:
api_key = "5c6ac9b9844cd60b13a5ea12c2eede98" # 발급받은 키만 복붙하세요.
request_date = "20220802" # 조회날짜를  xxxxyydd 20220802 형식으로 넣어주세요.
url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
print(url)

https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5c6ac9b9844cd60b13a5ea12c2eede98&targetDt=20220802


# json데이터를 팬더스 데이터프레임으로 변환

api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다.

쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데

파이썬의 딕셔너리와 거의 같다고 보시면 됩니다.

따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상

딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [11]:
# 위의 url 변수를 이용해 데이터를 요청합니다.
r = urllib.request.urlopen(url)

# 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
json_raw =r.read()

In [12]:
# b'~~~~~~~'로 시작하는 이 데이터는 파이썬에서 byte자료형으로 처리되고
# 데이터프레임으로는 바로 변환이 불가능합니다.
json_raw

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20220802~20220802","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20209343","movieNm":"\xed\x95\x9c\xec\x82\xb0: \xec\x9a\xa9\xec\x9d\x98 \xec\xb6\x9c\xed\x98\x84","openDt":"2022-07-27","salesAmt":"3418297205","salesShare":"56.5","salesInten":"-445775729","salesChange":"-11.5","salesAcc":"30676698831","audiCnt":"340707","audiInten":"-45277","audiChange":"-11.7","audiAcc":"2996787","scrnCnt":"2030","showCnt":"9659"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20205362","movieNm":"\xeb\xaf\xb8\xeb\x8b\x88\xec\x96\xb8\xec\xa6\x882","openDt":"2022-07-20","salesAmt":"843554512","salesShare":"13.9","salesInten":"-60441616","salesChange":"-6.7","salesAcc":"16439366347","audiCnt":"90863","audiInten":"-6344","audiChange":"-6.5","audiAcc":"1693084","scrnCnt":"945","showCnt":"2837"},{"r

In [13]:
# 바이트자료형임을 확인 
type(json_raw)

bytes

In [14]:
# 따라서 utf-8 형식으로 고쳐줘야 합니다. .decode()를 이용합니다.
# encode => 우리가 쓰는 문자를 컴퓨터가 쓰는 형태로 변환 
# decode => 컴퓨터가 쓰는 형태의 문자를 우리가 쓰는 형태로 변환
json_utf8 = json_raw.decode("utf-8")

In [15]:
# 딕셔너리 구조를 유지하고 있는 문자로 변환됨
type(json_utf8)

str

In [16]:
json_utf8

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20220802~20220802","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20209343","movieNm":"한산: 용의 출현","openDt":"2022-07-27","salesAmt":"3418297205","salesShare":"56.5","salesInten":"-445775729","salesChange":"-11.5","salesAcc":"30676698831","audiCnt":"340707","audiInten":"-45277","audiChange":"-11.7","audiAcc":"2996787","scrnCnt":"2030","showCnt":"9659"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20205362","movieNm":"미니언즈2","openDt":"2022-07-20","salesAmt":"843554512","salesShare":"13.9","salesInten":"-60441616","salesChange":"-6.7","salesAcc":"16439366347","audiCnt":"90863","audiInten":"-6344","audiChange":"-6.5","audiAcc":"1693084","scrnCnt":"945","showCnt":"2837"},{"rnum":"3","rank":"3","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20194376","movieNm":"탑건: 매버릭","openDt":"2022-06-22","salesAmt":"678742098","salesShare":"11.2","salesInten":"-52334711"

In [17]:
# json.loads(딕셔너리 형태의 문자열 자료)
# 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
json_complete = json.loads(json_utf8)

In [18]:
# 딕셔너리(json) 변환 완료 확인
type(json_complete)

dict

In [19]:
# json_normalize(딕셔너리 자료)
# 딕셔너리 자료를 데이터프레임 형태로 변환해줌
pd.json_normalize(json_complete) # 보통 json 데이터를 받아오면 list화 되어있는 컬럼이 하나 들어옴. 해당 list 데이터를 타켓해서 데이터 가져오기

,boxOfficeResult.boxofficeType,boxOfficeResult.showRange,boxOfficeResult.dailyBoxOfficeList
0,일별 박스오피스,20220802~20220802,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."


In [20]:
# boxOfficeResult.dailyBoxOfficeList 컬럼 내부에 딕셔너리가 여럿 묶인 리스트가 보이므로
# boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20209343,한산: 용의 출현,2022-07-27,3418297205,56.5,-445775729,-11.5,30676698831,340707,-45277,-11.7,2996787,2030,9659
1,2,2,0,OLD,20205362,미니언즈2,2022-07-20,843554512,13.9,-60441616,-6.7,16439366347,90863,-6344,-6.5,1693084,945,2837
2,3,3,0,OLD,20194376,탑건: 매버릭,2022-06-22,678742098,11.2,-52334711,-7.2,76662106293,66650,-4841,-6.8,7187516,802,2072
3,4,4,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,351444239,5.8,-19020699,-5.1,2356729812,38807,-1971,-4.8,251109,677,1440
4,5,5,0,OLD,20208446,외계+인 1부,2022-07-20,363649895,6.0,-40994799,-10.1,14968915621,36506,-3867,-9.6,1434570,712,1804
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,190209473,3.1,-155223,-0.1,17660574672,18263,43,0.2,1708584,412,752
6,7,7,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,81394059,1.3,5183642,6.8,4434849457,8088,617,8.3,427202,185,271
7,8,8,1,OLD,20211792,헌트,2022-08-10,29720000,0.5,-6703000,-18.4,93969000,4280,233,5.8,10688,17,17
8,9,9,-1,OLD,20196410,비상선언,2022-08-03,26522000,0.4,-46635300,-63.7,247169216,1681,-3204,-65.6,22329,5,7
9,10,10,0,OLD,20224662,토르: 러브 앤 썬더,2022-07-06,12040299,0.2,-151561,-1.2,29461697403,1082,1,0.1,2711863,38,51


In [21]:
# json_normalize()는 pd.DataFrame()으로 대체 가능합니다.
pd.DataFrame(json_complete)

,boxOfficeResult
boxofficeType,일별 박스오피스
dailyBoxOfficeList,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."
showRange,20220802~20220802


In [22]:
url

'https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5c6ac9b9844cd60b13a5ea12c2eede98&targetDt=20220802'

## DB에 데이터를 저장하는 이유

### API 크롤링으로 서비스 제공시 

### 사용자가 조회할때마다 매번 API서버에 요청을 넣어서 

### 데이터를 가져오면 호출횟수를 금방 소모하게 됩니다. 

### 따라서, 보통 API를 활용한 서비스는 미리 데이터를  DB에 적재해놓고 

### 사용자가 요청시 내 DB에 있는 데이터를 우선적으로 보여주고, 

### 없으면 외부서버에 요청해서 내 DB에 INSERT한 다음 다시 내 DB 데이터를 사용자에게 보여줍니다.

날짜 데이터는 28까지인 달(2월), 30일까지인 달, 31일까지인 달 들이 있어서

단순히 문자열에 1씩 더하거나 날짜처리를 하기도 어렵고

위의 처리를 다 했다고 해도 4년에 1번 윤달이 있습니다.(2월 29일)

그래서, 날짜는 전적으로 날짜데이터를 이용해 처리해야만 합니다.

파이썬에서는 datetime을 이용해 처리합니다.(자바는 java.util.Date를 써서 처리)

In [23]:
import time
import datetime  # 파이썬 날짜 자료형

In [24]:
start_datetime = datetime.datetime(2021, 8, 9) # 년 월 일 순으로 넣어줍니다.
# 형식 바꾸기
target_date = start_datetime.strftime('%Y%m%d') # 영진위 형식으로 변경   
print(target_date)
# 날짜 하루 올리기
next_datetime = start_datetime + datetime.timedelta(days=1) # 날짜 자료형을 이용하면 일수만 입력하면 월별 날짜를 알아서 계산해줌
next_datetime

20210809


datetime.datetime(2021, 8, 10, 0, 0)

In [28]:
# target_dates = ['20220805', '20220806', '20220807']
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 3일치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
target_dates = ['20220805', '20220806', '20220807']
for date in target_dates : 
    request_date = date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)

    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw =r.read()

    # 한글 형식으로 변환, str 자료형
    json_utf8 = json_raw.decode("utf-8")

    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
    json_complete = json.loads(json_utf8)

    # boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
    data = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    
    data.to_excel('movie_data%s.xlsx' % request_date)


In [31]:
# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고[pd.DataFrame()]
data0805 = pd.read_excel('data/movie_data20220805.xlsx') 
data0806 = pd.read_excel('data/movie_data20220806.xlsx') 
data0807 = pd.read_excel('data/movie_data20220807.xlsx') 
data_result = data0805.append([data0806, data0807], ignore_index=True) 
data_result

C:\Users\a\AppData\Local\Temp\ipykernel_20240\96729394.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_result = data0805.append([data0806, data0807], ignore_index=True)


,Unnamed: 0,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,0,1,1,1,OLD,20209343,한산: 용의 출현,2022-07-27,2565550184,41.6,530985656,26.1,37567514644,250627,41280,19.7,3692258,1530,6343
1,1,2,2,-1,OLD,20196410,비상선언,2022-08-03,2094794151,34.0,71338791,3.5,7693545482,213574,-8542,-3.8,794759,1718,7281
2,2,3,3,1,OLD,20194376,탑건: 매버릭,2022-06-22,504483247,8.2,148299938,41.6,77905758474,46942,12422,36.0,7305825,605,1238
3,3,4,4,-1,OLD,20205362,미니언즈2,2022-07-20,413915957,6.7,948579,0.2,17793954400,43092,-1769,-3.9,1837970,783,1595
4,4,5,5,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,167553618,2.7,-5960802,-3.4,2909201168,18088,-1181,-6.1,312147,537,779
5,5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,142017614,2.3,34703455,32.3,18014179742,13226,3017,29.6,1741846,265,432
6,6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,113278760,1.8,22370761,24.6,15283607796,11557,2267,24.4,1466378,407,576
7,7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,49992320,0.8,-4086959,-7.6,192969299,5266,-680,-11.4,20859,362,513
8,8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,27421500,0.4,3966180,16.9,4509942774,2517,232,10.2,434339,75,94
9,9,10,10,26,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,12398000,0.2,11939500,2604.0,28919650670,2433,2378,4323.6,2806159,20,20


In [33]:
print(type(data_result))

<class 'pandas.core.frame.DataFrame'>


In [46]:
# 강사님께서 진행한 풀이

# target_dates = ['20220805', '20220806', '20220807']
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 3일치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
target_dates = ['20220805', '20220806', '20220807']

merged_data = pd.DataFrame()
for date in target_dates : 
    request_date = date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)

    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw =r.read()

    # 한글 형식으로 변환, str 자료형
    json_utf8 = json_raw.decode("utf-8")

    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
    json_complete = json.loads(json_utf8)

    # boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['날짜'] = date
    
    merged_data = pd.concat([merged_data, box_result])

# 결과
merged_data


,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,1,OLD,20209343,한산: 용의 출현,2022-07-27,2565550184,41.6,530985656,26.1,37567514644,250627,41280,19.7,3692258,1530,6343,20220805
1,2,2,-1,OLD,20196410,비상선언,2022-08-03,2094794151,34.0,71338791,3.5,7693532482,213574,-8542,-3.8,794758,1718,7281,20220805
2,3,3,1,OLD,20194376,탑건: 매버릭,2022-06-22,504483247,8.2,148299938,41.6,77905758474,46942,12422,36,7305825,605,1238,20220805
3,4,4,-1,OLD,20205362,미니언즈2,2022-07-20,413915957,6.7,948579,0.2,17793954400,43092,-1769,-3.9,1837970,783,1595,20220805
4,5,5,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,167553618,2.7,-5960802,-3.4,2909201168,18088,-1181,-6.1,312147,537,779,20220805
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,142017614,2.3,34703455,32.3,18014179742,13226,3017,29.6,1741846,265,432,20220805
6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,113278760,1.8,22370761,24.6,15283607796,11557,2267,24.4,1466378,407,576,20220805
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,49992320,0.8,-4086959,-7.6,192969299,5266,-680,-11.4,20859,362,513,20220805
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,27421500,0.4,3966180,16.9,4509942774,2517,232,10.2,434339,75,94,20220805
9,10,10,26,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,12398000,0.2,11939500,2604,28919650670,2433,2378,4323.6,2806159,20,20,20220805


In [48]:
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 1년치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
start_datetime = datetime.datetime(2021, 8, 8) # 년 월 일 순으로 넣어줍니다.

# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고[pd.DataFrame()]
merged_data = pd.DataFrame()
for idx in range(365) : 
    # 날짜를 문자로 변경
    target_date = start_datetime.strftime('%Y%m%d')
    
    # 요청을 넣을 날짜
    request_date = target_date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
    
    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw =r.read()

    # 한글 형식으로 변환, str 자료형
    json_utf8 = json_raw.decode("utf-8")

    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
    json_complete = json.loads(json_utf8)

    # boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['날짜'] = date
    
    merged_data = pd.concat([merged_data, box_result])
    
    # 해당 날짜 데이터를 가져와 통합한 뒤에는 날짜를 하루 더해주기
    start_datetime = start_datetime + datetime.timedelta(days=1) # 날짜
    print("%s일차 자료 수집 완료" % idx)
    
    # 너무 자주 크롤링시 블록먹을 가능성이 있음
    time.sleep(1)
    
# 결과
merged_data


0일차 자료 수집 완료
1일차 자료 수집 완료
2일차 자료 수집 완료
3일차 자료 수집 완료
4일차 자료 수집 완료
5일차 자료 수집 완료
6일차 자료 수집 완료
7일차 자료 수집 완료
8일차 자료 수집 완료
9일차 자료 수집 완료
10일차 자료 수집 완료
11일차 자료 수집 완료
12일차 자료 수집 완료
13일차 자료 수집 완료
14일차 자료 수집 완료
15일차 자료 수집 완료
16일차 자료 수집 완료
17일차 자료 수집 완료
18일차 자료 수집 완료
19일차 자료 수집 완료
20일차 자료 수집 완료
21일차 자료 수집 완료
22일차 자료 수집 완료
23일차 자료 수집 완료
24일차 자료 수집 완료
25일차 자료 수집 완료
26일차 자료 수집 완료
27일차 자료 수집 완료
28일차 자료 수집 완료
29일차 자료 수집 완료
30일차 자료 수집 완료
31일차 자료 수집 완료
32일차 자료 수집 완료
33일차 자료 수집 완료
34일차 자료 수집 완료
35일차 자료 수집 완료
36일차 자료 수집 완료
37일차 자료 수집 완료
38일차 자료 수집 완료
39일차 자료 수집 완료
40일차 자료 수집 완료
41일차 자료 수집 완료
42일차 자료 수집 완료
43일차 자료 수집 완료
44일차 자료 수집 완료
45일차 자료 수집 완료
46일차 자료 수집 완료
47일차 자료 수집 완료
48일차 자료 수집 완료
49일차 자료 수집 완료
50일차 자료 수집 완료
51일차 자료 수집 완료
52일차 자료 수집 완료
53일차 자료 수집 완료
54일차 자료 수집 완료
55일차 자료 수집 완료
56일차 자료 수집 완료
57일차 자료 수집 완료
58일차 자료 수집 완료
59일차 자료 수집 완료
60일차 자료 수집 완료
61일차 자료 수집 완료
62일차 자료 수집 완료
63일차 자료 수집 완료
64일차 자료 수집 완료
65일차 자료 수집 완료
66일차 자료 수집 완료
67일차 자료 수집 완료
68일차 자료 수집 완료
69일차 자료 수집 완료
70일차 자료 수집 완료
71일차 자료 수집 완료
72

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,2000829380,56.5,38859180,2,16579074530,196672,4041,2.1,1712375,1639,6757,20220807
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,676797240,19.1,-127499190,-15.9,2942253950,63095,-12197,-16.2,281377,1033,3306,20220807
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,284206020,8.0,1506480,0.5,7511398580,30046,108,0.4,836805,777,1654,20220807
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,194489780,5.5,-28197330,-12.7,29457331370,18740,-2673,-12.5,2908679,540,924,20220807
4,5,5,0,OLD,20218875,극장판 도라에몽: 진구의 신공룡,2021-08-05,88917620,2.5,-4415810,-4.7,337195080,9837,-612,-5.9,38669,502,781,20220807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,6,1,OLD,20209654,헤어질 결심,2022-06-29,169174033,1.7,-32677140,-16.2,18385204948,15203,-2798,-15.5,1775050,266,426,20220807
6,7,7,-1,OLD,20208446,외계+인 1부,2022-07-20,123965118,1.3,-98324419,-44.2,15629862451,11690,-7177,-38,1496935,374,499,20220807
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,77727199,0.8,2170399,2.9,346253298,7903,248,3.2,36417,357,466,20220807
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,44635198,0.5,623298,1.4,4598589872,4122,13,0.3,442570,91,115,20220807


In [77]:
# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요.
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다.
# 컬럼 타입 object이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다.
# 현재 수치데이터도 전부 문자로 간주되고 있기 때문에 int로 자료형을 바꿔서 계산해야함
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int) 
movie_table = merged_data.pivot_table(index='movieNm', values=['audiCnt', 'scrnCnt'], aggfunc='sum')
movie_table['screenMean'] = movie_table['audiCnt'] / movie_table['scrnCnt']
movie_table.sort_values(by='audiCnt', ascending=False)

,audiCnt,scrnCnt,screenMean
movieNm,,,
범죄도시 2,12686014,81645,155.380170
스파이더맨: 노 웨이 홈,7541536,90668,83.177483
탑건: 매버릭,7448595,57863,128.728116
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
한산: 용의 출현,4597833,22656,202.941075
...,...,...,...
지지,334,1,334.000000
포 언투 어스,300,1,300.000000
맛있는 여동생,259,20,12.950000


In [76]:
# 강사님 풀이 , 따로 나누는 이유는 내 풀이와의 차이점을 알고 다시 봤을때
# "이때 이런 실수를 했구나"를 알기 위함

# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요.
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다.
# 컬럼 타입 object이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다.
# 현재 수치데이터도 전부 문자로 간주되고 있기 때문에 int로 자료형을 바꿔서 계산해야함
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int) 
movie_table = merged_data.pivot_table(index='movieNm', values=['audiCnt', 'scrnCnt'], aggfunc='sum')
movie_table['screenMean'] = movie_table['audiCnt'] / movie_table['scrnCnt']
movie_table.sort_values(by='audiCnt', ascending=False)

,audiCnt,scrnCnt,screenMean
movieNm,,,
범죄도시 2,12686014,81645,155.380170
스파이더맨: 노 웨이 홈,7541536,90668,83.177483
탑건: 매버릭,7448595,57863,128.728116
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
한산: 용의 출현,4597833,22656,202.941075
...,...,...,...
지지,334,1,334.000000
포 언투 어스,300,1,300.000000
맛있는 여동생,259,20,12.950000
